In [26]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import os
import json
import folium
from folium import plugins
import re
from haversine import haversine, Unit
from datetime import datetime, date, time
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup
from collections import namedtuple
import geopandas as gpd
from folium.plugins import HeatMap

### Data from Bixi

In [27]:
data = pd.read_pickle("paths")

In [28]:
data

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance
0,4000,4000,101,1174.811881,45.549598,-73.541874,45.549598,-73.541874,0.000000
1,4000,4002,1,1864.000000,45.549598,-73.541874,45.516937,-73.640483,8.496129
2,4000,5007,1,3585.000000,45.549598,-73.541874,45.523319,-73.520127,3.377533
3,4000,6001,52,1414.403846,45.549598,-73.541874,45.510351,-73.556508,4.510499
4,4000,6002,93,506.064516,45.549598,-73.541874,45.539385,-73.541000,1.137631
...,...,...,...,...,...,...,...,...,...
275730,7149,7121,1,2250.000000,NaN,NaN,NaN,NaN,NaN
275731,7149,7122,1,1336.000000,NaN,NaN,NaN,NaN,NaN
275732,7149,7137,1,1091.000000,NaN,NaN,NaN,NaN,NaN
275733,7149,7138,1,1697.000000,NaN,NaN,NaN,NaN,NaN


In [29]:
data = data.sort_values(by="counts", axis=0)

In [30]:
data

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance
137867,6376,6411,1,2181.000000,45.552842,-73.565295,45.513303,-73.572961,4.436990
218356,7147,7047,1,2477.000000,45.534739,-73.620769,45.493909,-73.559507,6.587714
218364,7147,7068,1,1887.000000,45.534739,-73.620769,45.487531,-73.565070,6.811214
64354,6163,6739,1,1846.000000,45.535430,-73.582200,45.580315,-73.535314,6.183473
218369,7147,7076,1,2329.000000,45.534739,-73.620769,45.491230,-73.580782,5.754533
...,...,...,...,...,...,...,...,...,...
52252,6136,6163,2089,334.380086,45.527616,-73.589115,45.535430,-73.582200,1.022253
15073,6036,6036,2355,2087.896391,45.504242,-73.553469,45.504242,-73.553469,0.000000
21831,6052,6406,2385,1380.174843,45.497515,-73.552571,45.480208,-73.577599,2.740398
11836,6026,6026,2815,1848.837300,45.507610,-73.551836,45.507610,-73.551836,0.000000


In [31]:
data = data.dropna()

In [62]:
data = data[data['distance']>0.]

In [63]:
data

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance
137867,6376,6411,1,2181.000000,45.552842,-73.565295,45.513303,-73.572961,4.436990
218356,7147,7047,1,2477.000000,45.534739,-73.620769,45.493909,-73.559507,6.587714
218364,7147,7068,1,1887.000000,45.534739,-73.620769,45.487531,-73.565070,6.811214
64354,6163,6739,1,1846.000000,45.535430,-73.582200,45.580315,-73.535314,6.183473
218369,7147,7076,1,2329.000000,45.534739,-73.620769,45.491230,-73.580782,5.754533
...,...,...,...,...,...,...,...,...,...
52254,6136,6165,1992,238.057229,45.527616,-73.589115,45.532514,-73.584811,0.639508
143719,6396,6398,2010,267.697512,45.554214,-73.551560,45.551192,-73.540872,0.897440
21547,6052,6026,2080,887.340385,45.497515,-73.552571,45.507610,-73.551836,1.123981
52252,6136,6163,2089,334.380086,45.527616,-73.589115,45.535430,-73.582200,1.022253


In [64]:
data[(data['counts']>=20)&(data['counts']<=30)]

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance
172319,6906,6397,20,1575.350000,45.531955,-73.598160,45.553219,-73.539782,5.124509
108926,6264,6360,20,538.700000,45.542570,-73.592144,45.553215,-73.587520,1.237261
32931,6085,6111,20,1342.700000,45.495200,-73.563280,45.526200,-73.549266,3.615914
51068,6132,6259,20,651.350000,45.529135,-73.559234,45.540881,-73.584157,2.339774
202638,7067,6031,20,964.000000,45.506251,-73.571391,45.504263,-73.571523,0.221309
...,...,...,...,...,...,...,...,...,...
40294,6104,6398,30,1370.266667,45.516818,-73.554188,45.551192,-73.540872,3.960484
81794,6201,6207,30,841.933333,45.523544,-73.587332,45.512832,-73.576888,1.442599
97771,6234,6331,30,834.233333,45.527410,-73.603935,45.543424,-73.632462,2.847338
47294,6122,7011,30,516.900000,45.530570,-73.549130,45.520574,-73.556144,1.238573


In [65]:
data.sample(150)

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance
145692,6405,6129,1,1748.000000,45.482098,-73.574876,45.529971,-73.563053,5.402353
24099,6060,6419,1,2929.000000,45.501367,-73.565837,45.491513,-73.633649,5.397809
139919,6385,6088,15,2105.133333,45.536110,-73.576287,45.489951,-73.567091,5.182435
214286,7124,6223,19,912.263158,45.516783,-73.611423,45.517000,-73.589000,1.747210
198562,7052,6137,2,1794.000000,45.483006,-73.562187,45.530347,-73.566988,5.277418
...,...,...,...,...,...,...,...,...,...
12705,6027,6904,15,1787.000000,45.502054,-73.573465,45.542060,-73.597873,4.837824
95938,6230,6908,85,520.764706,45.523856,-73.600127,45.533410,-73.601124,1.065230
156512,6703,6225,6,2200.000000,45.555004,-73.555355,45.520188,-73.590559,4.744026
135450,6370,6321,3,1675.333333,45.557192,-73.569847,45.535900,-73.622442,4.730841


### Data from the bike paths

In [66]:
fname = "./BikePaths/reseau_cyclable_2018_c.geojson"
bp = gpd.read_file(fname)
bp = bp[bp['Ville_MTL']=='OUI']
bp = bp.drop(columns=['ID', 'ID_TRC_GEO', 'TYPE_VOIE', 'TYPE_VOIE2', 'SEPARATEUR', 'SAISONS4', 'PROTEGE_4S', 'Ville_MTL'])
bp = bp.reset_index(drop=True)

In [67]:
bp['distances'] = bp.apply(lambda x: np.array([]),axis=1)
for k in range(len(bp)):
    distances = []
    for l in range(len(bp['geometry'][k])):
        subdistances = np.array([])
        for m in range(len(bp['geometry'][k][l].xy[0])-1):
            #distances[l] = np.append(distances[l], haversine((x.latitude_start,x.longitude_start),(x.latitude_end,x.longitude_end)))
            subdistances = np.append(subdistances, 1000*haversine((bp['geometry'][k][l].xy[1][m], bp['geometry'][k][l].xy[0][m]),(bp['geometry'][k][l].xy[1][m+1], bp['geometry'][k][l].xy[0][m+1])))
        distances.append(subdistances)
    bp['distances'][k] = np.array(distances)

C:\Users\quent\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


### Useful functions

In [68]:
def isincluded(x, y, a, b, da, db): # Attention ! da and db must be positive
    if abs(2*x - 2*a - da) <= da:
        if abs(2*y - 2*b - db) <= db:
            return 1
    return 0

In [69]:
def calculate_truncated_distance(x1, y1, x2, y2, a, b, da, db, d0, d1): # Attention ! da and db must be positive
    # Case where the first point is in the square
    if ((d0 == 1) and (d1 ==0)):
        if x1==x2:
            if y2<b:
                xt, yt = x1, b
            else:
                xt, yt = x1, b + db
        elif y1==y2:
            if x2<a:
                xt, yt = a, y1
            else:
                xt, yt = a + da, y1
        else:
            # Case y2 < b (under the square)
            if y2 < b:
                xtest = (x2-x1)*(b-y1)/(y2-y1) + x1
                if xtest < a:
                    xt, yt = a, (y2-y1)*(a-x1)/(x2-x1) + y1
                elif xtest > a + da:
                    xt, yt = a+da, (y2-y1)*(a+da-x1)/(x2-x1) + y1
                else : 
                    xt, yt = xtest, b
            # Case y2 > b + db (above the square)
            elif y2 > b+db:
                xtest = (x2-x1)*(b+db-y1)/(y2-y1) + x1
                if xtest < a:
                    xt, yt = a, (y2-y1)*(a-x1)/(x2-x1) + y1
                elif xtest > a + da:
                    xt, yt = a+da, (y2-y1)*(a+da-x1)/(x2-x1) + y1
                else : 
                    xt, yt = xtest, b+db
            # Case y in the range(b, b+db)
            else:
                if x2 <a:
                    xt, yt = a, (y2-y1)*(a-x1)/(x2-x1) + y1
                else: # Case x2>a+da
                    xt, yt = a+da, (y2-y1)*(a+da-x1)/(x2-x1) + y1
        return  1000*haversine((y1,x1),(yt,xt))
    elif ((d0 == 0) and (d1 ==1)):
        return calculate_truncated_distance(x2, y2, x1, y1, a, b, da, db, d1, d0)
    else:
        return(0.)

In [70]:
def compute_density_in_squares(xp1,yp1,xp2,yp2):
    '''(x1,y1) represents the point at the bottom left and (x2,y2) the point at the top right
    The n_x_squares number represents the number of squares between x1 and x2. 
    The length of a square according to x will then be used to calculate the number of squares according to y. '''
    
    # The point (x1, y1) must correspond to the point at the bottom left of the square.
    if xp1<=xp2:
        x1, x2 = xp1, xp2
    else:
        x1, x2 = xp2, xp1
    if yp1<=yp2:
        y1, y2 = yp1, yp2
    else:
        y1, y2 = yp2, yp1
    # We look at whether the longest length of the square is according to x or y.
    if x2-x1>y2-y1:
        y1, y2 = (y1+y2)/2 - (x2-x1)/2 , (y1+y2)/2 + (x2-x1)/2
    else:
        x1, x2 = (x1+x2)/2 - (y2-y1)/2 , (x1+x2)/2 + (y2-y1)/2
    # Convert the coordinates to a and b
    a, b = x1, y1
    da, db = x2-x1, y2-y1 # da = db
    # Calculate the total distance in this square
    total_distance_in_the_square = 0.
    for k in range(len(bp['geometry'])):
        for l in range(len(bp['geometry'][k])): # l et k sont les id d'un chemin simple
            for i in range(len(bp['geometry'][k][l].xy[0])-1): # i est l'id du point sur ce chemin
                x1, y1, x2, y2 = bp['geometry'][k][l].xy[0][i], bp['geometry'][k][l].xy[1][i] , bp['geometry'][k][l].xy[0][i+1], bp['geometry'][k][l].xy[1][i+1]
                d0, d1 = isincluded(x1, y1, a, b, da, db) , isincluded(x2, y2, a, b, da, db)
                if d0+d1==2: # Si les deux points sont dans le carré, on ajoute le premier
                    total_distance_in_the_square += bp['distances'][k][l][i]
                elif ((d1==1) or (d0==1)): # Si seul le deuxième point est dans le carré, on ajoute l'intersection seulement
                    total_distance_in_the_square += calculate_truncated_distance(x1, y1, x2, y2, a, b, da, db, d0, d1)
    total_distance_in_the_square = (total_distance_in_the_square*1e-3)/((haversine((b, a), (b+db, a)))**2) # unit : km / (km**2)
    return total_distance_in_the_square

In [71]:
subdata = data.sample(3)
subdata = subdata.reset_index(drop=True)

In [72]:
subdata['bp_density'] = bp.apply(lambda x: 0.,axis=1)

In [73]:
subdata

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance,bp_density
0,6324,6278,4,1391.750000,45.545604,-73.634740,45.550010,-73.59576,3.074752,0.0
1,7029,6252,5,1102.600000,45.538021,-73.583524,45.533180,-73.61544,2.543521,0.0
2,6354,6435,94,808.925532,45.471743,-73.613924,45.481291,-73.60033,1.500228,0.0


In [74]:
import time
start_time = time.time()
for k in subdata.index:
    subdata['bp_density'][k] = compute_density_in_squares(subdata['longitude_start'][k],subdata['latitude_start'][k],subdata['longitude_end'][k],subdata['latitude_end'][k])
elapsed_time = time.time() - start_time
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

C:\Users\quent\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


'00:00:25'

In [75]:
subdata

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance,bp_density
0,6324,6278,4,1391.750000,45.545604,-73.634740,45.550010,-73.59576,3.074752,2.150715
1,7029,6252,5,1102.600000,45.538021,-73.583524,45.533180,-73.61544,2.543521,3.195246
2,6354,6435,94,808.925532,45.471743,-73.613924,45.481291,-73.60033,1.500228,1.090793


In [76]:
number_of_values = data.apply(lambda x: x.count(), axis=0)

In [77]:
number_of_values


start_station_code    218563
end_station_code      218563
counts                218563
duration              218563
latitude_start        218563
longitude_start       218563
latitude_end          218563
longitude_end         218563
distance              218563
dtype: int64

In [78]:
len(data[(data['counts']>=0)&(data['counts']<10)])

126795

In [79]:
len(data[(data['counts']>=10)&(data['counts']<20)])

33917

In [80]:
len(data[(data['counts']>=20)&(data['counts']<40)])

27402

In [81]:
len(data[(data['counts']>=40)&(data['counts']<100)])

21834

In [82]:
len(data[(data['counts']>=100)&(data['counts']<500)])

8186

In [83]:
len(data[(data['counts']>=500)&(data['counts']<1000)])

330

In [84]:
len(data[(data['counts']>=1000)&(data['counts']<2000)])

95

In [85]:
len(data[(data['counts']>=2000)])

4

In [86]:
data[(data['counts']>=10)&(data['counts']<20)].sample(3)

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance
67107,6169,6334,16,1410.375000,45.527432,-73.579917,45.543712,-73.628284,4.179629
9401,6019,7065,11,1492.818182,45.517999,-73.568184,45.543651,-73.605651,4.080848
76749,6190,6910,17,680.117647,45.527009,-73.585766,45.537041,-73.602026,1.687727


In [90]:
bixi_subdata = data[(data['counts']>=2000)].sample(3)

cut_counts_list = [0, 10, 20, 40, 100, 500, 1000, 2000]
for k in range(len(cut_counts_list)-1):
    bixi_subdata = pd.concat([bixi_subdata, data[(data['counts']>=cut_counts_list[k])&(data['counts']<cut_counts_list[k+1])].sample(10)])

    

In [91]:
bixi_subdata

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance
52252,6136,6163,2089,334.380086,45.527616,-73.589115,45.535430,-73.582200,1.022253
21547,6052,6026,2080,887.340385,45.497515,-73.552571,45.507610,-73.551836,1.123981
21831,6052,6406,2385,1380.174843,45.497515,-73.552571,45.480208,-73.577599,2.740398
213529,7116,6730,1,1733.000000,45.596796,-73.535179,45.570081,-73.573047,4.184379
13467,6031,7099,1,2032.000000,45.504263,-73.571523,45.497047,-73.643216,5.644820
...,...,...,...,...,...,...,...,...,...
43973,6114,6110,1163,308.029235,45.523530,-73.551990,45.529033,-73.546348,0.753393
21554,6052,6036,1256,681.863057,45.497515,-73.552571,45.504242,-73.553469,0.751328
52553,6136,7026,1061,179.029218,45.527616,-73.589115,45.527041,-73.593471,0.345296
60347,6154,6187,1340,307.902239,45.532290,-73.575440,45.525628,-73.581775,0.890077


In [92]:
bixi_subdata = bixi_subdata.reset_index(drop=True)

In [94]:
bixi_subdata['bp_density'] = bixi_subdata.apply(lambda x: 0.,axis=1)

In [95]:
bixi_subdata

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance,bp_density
0,6136,6163,2089,334.380086,45.527616,-73.589115,45.535430,-73.582200,1.022253,0.0
1,6052,6026,2080,887.340385,45.497515,-73.552571,45.507610,-73.551836,1.123981,0.0
2,6052,6406,2385,1380.174843,45.497515,-73.552571,45.480208,-73.577599,2.740398,0.0
3,7116,6730,1,1733.000000,45.596796,-73.535179,45.570081,-73.573047,4.184379,0.0
4,6031,7099,1,2032.000000,45.504263,-73.571523,45.497047,-73.643216,5.644820,0.0
...,...,...,...,...,...,...,...,...,...,...
68,6114,6110,1163,308.029235,45.523530,-73.551990,45.529033,-73.546348,0.753393,0.0
69,6052,6036,1256,681.863057,45.497515,-73.552571,45.504242,-73.553469,0.751328,0.0
70,6136,7026,1061,179.029218,45.527616,-73.589115,45.527041,-73.593471,0.345296,0.0
71,6154,6187,1340,307.902239,45.532290,-73.575440,45.525628,-73.581775,0.890077,0.0


In [96]:
import time
start_time = time.time()
for k in bixi_subdata.index:
    bixi_subdata['bp_density'][k] = compute_density_in_squares(bixi_subdata['longitude_start'][k],bixi_subdata['latitude_start'][k],bixi_subdata['longitude_end'][k],bixi_subdata['latitude_end'][k])
elapsed_time = time.time() - start_time
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

C:\Users\quent\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


'00:00:23'

In [97]:
bixi_subdata

,start_station_code,end_station_code,counts,duration,latitude_start,longitude_start,latitude_end,longitude_end,distance,bp_density
0,6136,6163,2089,334.380086,45.527616,-73.589115,45.535430,-73.582200,1.022253,3.467586
1,6052,6026,2080,887.340385,45.497515,-73.552571,45.507610,-73.551836,1.123981,1.337192
2,6052,6406,2385,1380.174843,45.497515,-73.552571,45.480208,-73.577599,2.740398,2.362488
3,7116,6730,1,1733.000000,45.596796,-73.535179,45.570081,-73.573047,4.184379,0.000000
4,6031,7099,1,2032.000000,45.504263,-73.571523,45.497047,-73.643216,5.644820,0.000000
...,...,...,...,...,...,...,...,...,...,...
68,6114,6110,1163,308.029235,45.523530,-73.551990,45.529033,-73.546348,0.753393,0.000000
69,6052,6036,1256,681.863057,45.497515,-73.552571,45.504242,-73.553469,0.751328,0.000000
70,6136,7026,1061,179.029218,45.527616,-73.589115,45.527041,-73.593471,0.345296,0.000000
71,6154,6187,1340,307.902239,45.532290,-73.575440,45.525628,-73.581775,0.890077,0.000000
